In [123]:
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd

import glob, os, gc, datetime, io

import json
import matplotlib as mpl
import pylab as plt

from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ( ColumnDataSource, GeoJSONDataSource, 
                           LinearColorMapper, ColorBar, HoverTool, 
                           LogColorMapper, LogTicker )
from bokeh.plotting import figure
from bokeh.palettes import brewer

import panel as pn
import panel.widgets as pnw

%matplotlib inline 

In [124]:
gc.collect()

1191

In [125]:
RU_idr = pd.read_csv('./Таблица_2020-11-13_20-16.csv')
RU_idr['Дата'] = pd.to_datetime( RU_idr['Дата'] , format='%d.%m.%Y' )
RU_idr['Активных'] = RU_idr['Заражений'] - RU_idr['Смертей'] - RU_idr['Выздоровлений']
RU_idr['Активных на 100К населения'] = (10**5*RU_idr['Активных']/RU_idr['Регион население']).round(2)
RU_idr['Смертей на 100К населения'] = (10**5*RU_idr['Смертей']/RU_idr['Регион население']).round(2)
RU_idr['Заражений за день на 100К населения'] = (10**5*RU_idr['Заражений за день']/RU_idr['Регион население']).round(2)
RU_idr['Смертей за день на 100К населения'] = (10**5*RU_idr['Смертей за день']/RU_idr['Регион население']).round(2)
RU_idr['Летальность, %'] = RU_idr['Летальность, %'].round(2)

RU_idr_i = RU_idr.set_index(['Регион название',"Дата"])

In [27]:
#rureg_map = gpd.read_file("RU_regions_ya_index.shp", encoding = 'cp1251')
#rureg_map.set_index('NL_NAME_1', inplace = True)
rureg_map = gpd.read_file('./ru_regions_ya.geojson')
rureg_map.set_index('index', inplace = True)

In [28]:
rureg_map = rureg_map[['full_name','geometry']]

In [126]:
def get_dataset(key=False,date=False):
    if not key:
        key = 'Заражений'
    if not date:
        date = RU_idr['Дата'].max().strftime('%Y-%m-%d')
    df = RU_idr_i[key].unstack()[date]
    #merge with the geopandas dataframe
    merged = rureg_map.copy()
    merged['value'] = df.loc[merged.index].values
    #merged['value'] = merged['value'].astype(float).round(2)
    merged['region'] = merged.index.values
    #merged[key] = merged[key].fillna(0)    
    return merged, key, date

In [127]:
def get_geodatasource(gdf):    
    """Get getjsondatasource from geopandas object"""
    geosource = GeoJSONDataSource(geojson = gdf.to_json())
    return geosource

In [128]:
def bokeh_plot_map(gdf, title='', pal = 'RdBu'): #'OrRd'
    """Plot bokeh map from GeoJSONDataSource """
    gdf, key, date = gdf
    geosource = get_geodatasource(gdf)
    palette = brewer[pal][10]
    #palette = palette[::-1]
    vals = gdf['value']
    if gdf['value'].min() == 0:
        gdf['value'] = gdf['value']+1
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    
    if np.abs(gdf['value'].skew()) < 3:
        color_mapper = LinearColorMapper(palette = palette, low = vals.min(), high = vals.max())
        color_bar = ColorBar(color_mapper=color_mapper, #label_standoff=10, 
                         width=500, height=20, 
                         location=(0,0), orientation='horizontal')

    else:
        color_mapper = LogColorMapper(palette = palette, low = vals.min(), high = vals.max() )
        color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(base = 10),# label_standoff=100, 
                         width=500, height=20, 
                         location=(0,0), orientation='horizontal')

    p = figure(title = f'{key}, {date}', 
               plot_height = 600 ,
               plot_width = 950, 
               tools = "pan, wheel_zoom, box_zoom, reset, save",
               toolbar_location='right')
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    #Add patch renderer to figure
    regs = p.patches('xs','ys', 
                     source=geosource, 
                     fill_alpha=1, line_width=0.1, 
                     line_color='gray',  
                     fill_color={'field' : 'value' ,
                                 'transform': color_mapper})
    #Specify figure layout.
    p.add_layout(color_bar, 'below')
    
    p.add_tools(HoverTool(renderers = [regs],
                          tooltips = [('Регион', '@region'),
                                      ( key,     '@value')]))
    p.xaxis.visible = False
    p.yaxis.visible = False

    return p

In [137]:
def map_dash():
    """Map dashboard"""
    from bokeh.models.widgets import DataTable
    stdate = datetime.date.fromisoformat(RU_idr['Дата'].min().strftime('%Y-%m-%d'))
    endate = datetime.date.fromisoformat(RU_idr['Дата'].max().strftime('%Y-%m-%d'))
    map_pane = pn.pane.Bokeh(width=950)
    data_select = pnw.Select(name='Показатель', options=['Регион население',
                                                         'Заражений',
                                                         'Выздоровлений', 
                                                         'Смертей',
                                                         'Смертей за день',
                                                         'Заражений за день',
                                                         'Выздоровлений за день',
                                                         'Заражений на 100К населения',
                                                         'Летальность, %', 
                                                         'Активных',
                                                         'Активных на 100К населения', 
                                                         'Смертей на 100К населения',
                                                         'Заражений за день на 100К населения',
                                                         'Смертей за день на 100К населения'],
                             value = 'Заражений')
    date_select = pnw.DatePicker(name = 'Дата', end=endate, start=stdate, 
                                 value=endate    )
    #finput = pnw.FileInput()
    #pal_select = pnw.Select(name='Палитра', options = ['RdBu', 'OrRd'], value = 'RdBu')
    
    def update_map(event):
        #print ([data_select.value, date_select.value])
        gdf = get_dataset(data_select.value, pd.to_datetime(date_select.value).strftime('%Y-%m-%d'))        
        #print (gdf[0].head())        
        map_pane.object = bokeh_plot_map(gdf)#, pal = pal_select.value)        
        return
    
    date_select.param.watch(update_map,'value')
    date_select.param.trigger('value')
    data_select.param.watch(update_map,'value')
    #pal_select.param.watch(update_map,'value')
    app = pn.Column(pn.Row(data_select,date_select),map_pane)
    return app

In [134]:
pn.extension()
output_notebook()

Loading BokehJS ...

In [135]:
app = map_dash()

In [136]:
app

Traceback (most recent call last):
 File "C:\Users\Sammael\anaconda3\envs\geo_env\lib\site-packages\pyviz_comms\__init__.py", line 316, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\Sammael\anaconda3\envs\geo_env\lib\site-packages\panel\viewable.py", line 259, in _on_msg
 doc.unhold()
 File "C:\Users\Sammael\anaconda3\envs\geo_env\lib\site-packages\bokeh\document\document.py", line 668, in unhold
 self._trigger_on_change(event)
 File "C:\Users\Sammael\anaconda3\envs\geo_env\lib\site-packages\bokeh\document\document.py", line 1151, in _trigger_on_change
 self._with_self_as_curdoc(event.callback_invoker)
 File "C:\Users\Sammael\anaconda3\envs\geo_env\lib\site-packages\bokeh\document\document.py", line 1169, in _with_self_as_curdoc
 return f()
 File "C:\Users\Sammael\anaconda3\envs\geo_env\lib\site-packages\bokeh\util\callback_manager.py", line 155, in invoke
 callback(attr, old, new)
 File "C:\Users\Sammael\anaconda3\envs\geo_env\lib\site-packages\panel\reactive.py", line 216, in _comm_change
 self._process_events({attr: new})
 File "C:\Users\Sammael\anaconda3\envs\geo_env\lib\site-packages\panel\reactive.py", line 187, in _process_events
 self.param.set_param(**self._process_property_change(events))
 File "C:\Users\Sammael\anaconda3\envs\geo_env\lib\site-packages\param\parameterized.py", line 1451, in set_param
 self_._batch_call_watchers()
 File "C:\Users\Sammael\anaconda3\envs\geo_env\lib\site-packages\param\parameterized.py", line 1578, in _batch_call_watchers
 watcher.fn(*events)
 File "<ipython-input-133-10c6fe93e96c>", line 33, in update_map
 RU_idr = pd.read_csv(finput)
 File "C:\Users\Sammael\anaconda3\envs\geo_env\lib\site-packages\pandas\io\parsers.py", line 686, in read_csv
 return _read(filepath_or_buffer, kwds)
 File "C:\Users\Sammael\anaconda3\envs\geo_env\lib\site-packages\pandas\io\parsers.py", line 434, in _read
 fp_or_buf, _, compression, should_close = get_filepath_or_buffer(
 File "C:\Users\Sammael\anaconda3\envs\geo_env\lib\site-packages\pandas\io\common.py", line 243, in get_filepath_or_buffer
 raise ValueError(msg)
ValueError: Invalid file path or buffer object type: <class 'panel.widgets.input.FileInput'>

Column
    [0] Row
        [0] Select(name='Показатель', options=['Регион население', ...], value='Заражений')
        [1] DatePicker(end=datetime.date(2020, ..., name='Дата', start=datetime.date(2020, ..., value=datetime.date(2020, ...)
        [2] FileInput()
    [1] Bokeh(Figure, width=950)

In [107]:
gc.collect()

40